In [90]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import time

In [91]:
url = f"https://baseballsavant.mlb.com/savant-player/{player_id}?stats=statcast-r-running-mlb"

# Send GET request to fetch the page content
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Failed to fetch page: {response.status_code}")

# Parse the page content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

rows = soup.find(id='stats-section').find_all(class_='table-savant')[-2].find_all('tr')

data = {'Year': [], 'Sprint': []}

for i in rows[1:]:
    
    data['Year'].append(i.find_all('td')[0].get_text())
    
    data['Sprint'].append(i.find_all('td')[2].get_text())

In [86]:
# Replace 'your_directory_path' with the path to your folder
folder_path = 'Runner_final'

# List all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]


In [87]:
for player in csv_files:
    data_df = pd.read_csv(f'{folder_path}/{player}')
    
    data_df['Year'] = data_df['Date'].apply(lambda x: int(x.split('-')[0]))
    
    if len(data_df) == 0:
        continue
    
    player_id = data_df['runner_id'].tolist()[0]
    
    url = f"https://baseballsavant.mlb.com/savant-player/{player_id}?stats=statcast-r-running-mlb"

    # Send GET request to fetch the page content

    response = requests.get(url)
    if response.status_code != 200:
        time.sleep(5)
        response = requests.get(url)
    # Parse the page content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    rows = soup.find(id='stats-section').find_all(class_='table-savant')[-2].find_all('tr')

    data = {'Current_Year': [], 'Sprint': []}

    for i in rows[1:]:
        
        if len(i.find_all('td')) == 0:
            break

        data['Current_Year'].append(int(i.find_all('td')[0].get_text()))

        data['Sprint'].append(float(i.find_all('td')[2].get_text()))
        
    player_data = pd.DataFrame(data)

    player_data['Year'] = player_data['Current_Year'].shift(1)
    
    final_df = pd.merge(data_df, player_data, on='Year')
    
    final_df.to_csv(f'Runner_final/{player}')

In [65]:
requests.get(url)

<Response [200]>

In [66]:
response

<Response [500]>

In [97]:
# Replace 'your_directory_path' with the path to your folder
folder_path = 'catchers'

# List all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]


In [98]:
for player in csv_files:
    data_df = pd.read_csv(f'{folder_path}/{player}')
    
    if len(data_df) == 0:
        continue
    
    data_df['Year'] = data_df['Date'].apply(lambda x: int(x.split('-')[0]))
    
    player_id = data_df['credit_id'].tolist()[0]
    
    url = f"https://baseballsavant.mlb.com/savant-player/{player_id}?stats=statcast-r-catching-mlb"
    
    response = requests.get(url)

    # Parse the page content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    if len(soup.find_all(id='statcastCatching')) == 0:
        continue
        
    try:

        rows = soup.find(id='statcastCatching').find_all(class_='table-savant')[1].find('tbody').find_all('tr')

        data = {'Current_Year': [], 'Arm_Strength': [], 'Exchange': []}
        
    except:
        continue

    for i in rows[1:]:
        
        if len(i.find_all('td')) == 0:
            break

        data['Current_Year'].append(int(i.find_all('td')[0].get_text()))

        data['Arm_Strength'].append(i.find_all('td')[2].get_text())

        data['Exchange'].append(i.find_all('td')[3].get_text())
        
    player_data = pd.DataFrame(data)

    player_data['Year'] = player_data['Current_Year'].shift(1)
    
    final_df = pd.merge(data_df, player_data, on='Year')
    
    final_df.to_csv(f'catchers/{player}')

In [99]:
url

'https://baseballsavant.mlb.com/savant-player/572180?stats=statcast-r-catching-mlb'

In [94]:
soup.find(id='statcastCatching')

<div class="statcast-group" id="statcastCatching" style="display:none;">
<div id="statcastCatchingBlocksAboveAverage"></div>
<div id="statcastCatchingCSAboveAverage"></div>
</div>

In [47]:
len(soup.find_all(id='statcastCatching'))

0

In [47]:
player_name = pd.read_csv('catcher_data/A.J._Ellis.csv')['credit'].tolist()[0]

In [48]:
import requests

def get_player_id(player_name):
    base_url = "https://statsapi.mlb.com/api/v1/people/search"
    response = requests.get(base_url, params={"name": player_name})
    
    if response.status_code == 200:
        data = response.json()
        if "people" in data and data["people"]:
            return data["people"][0]["id"]
        else:
            return f"No player found with name: {player_name}"
    else:
        return f"Error: {response.status_code}"

player_id = get_player_id(player_name)
print(f"Player ID for {player_name}: {player_id}")


Player ID for A.J. Ellis: 457705
